# data cleaning

turn tweet data set into workable dataframe using pyspark

In [1]:
from pyspark import SparkContext

In [2]:
try:
    sc.stop()
    sc = SparkContext('local')
except NameError:
    sc = SparkContext('local')

In [8]:
tweets = sc.textFile("file:///Users/tijsvandenheuvel/bin/tweets/dev.txt")

In [47]:
def val(x):
    if x == '<START>':
        val.i+=1
    return val.i
val.i = -1

In [152]:
tweets2 = tweets.flatMap(lambda x: x.split(' ')).filter(lambda x:x!='O').filter(lambda x:x!='').filter(lambda x:x!='<STOP>').map(lambda x:[val(x),x]).filter(lambda x:x[1]!='<START>')

text_rdd = tweets2.filter(lambda x: x[1][0]!=':').reduceByKey(lambda x,y: x+' '+y)

emoji_rdd = tweets2.filter(lambda x: x[1][0]==':').reduceByKey(lambda x,y:x+' '+y)

combined = text_rdd.leftOuterJoin(emoji_rdd).map(lambda x:(x[0],x[1][0],x[1][1]))

In [155]:
from pyspark.sql import SQLContext

s   = SQLContext(sc)
df = s.createDataFrame(combined, ['id', 'text','emoji'])

df.write.csv('file:///Users/tijsvandenheuvel/bin/tweets/full', sep=',', header=True)